Docker

    Agenda


           introduktion til Docker
           Teknik
           Community

    Nye tiltag
    
           Docker-machine
           Docker-compose (fig)
           Docker-swarm
           
           privat Docker registry
           
      Demo
          
    Indledning 1
        
        Hvem er jeg

        
    Hvad er Docker ? 10
        Eksempler
        
        Use Cases
            Rapid deployment
            Kontrolleret udviklingsmiljø
            Skalerbarhed
    Hvordan kan det lade sig gøre? (Teknologien bag) 5
        namespaces
        cgroups
        linuxcontainers
        Aufs
        Docker daemon
        Docker networking
    Hvordan kommer jeg igang? 10
        Docker registry
        docker ps
        Docker run
        Docker commit
        Docker docker build
            Dockerfile
                ONBUILD
        Docker exec
        DOcker logs
    Tricks og værktøjer 5
        Fig
        phusion/baseimage
        
    Seje applikationer 4
       torrent-docker
       ECS
       
    Mangler, udviklings områder 4
        Orkestrering
        Monitorering
        Security
        

# IDA Embedded - Docker


24/2/2015

Anders Lehmann

anders@ase.au.dk

**Aarhus School of Engineering, AU Herning**

# Agenda

- Indledning     
    - Hvad er Docker?
    - Hvordan virker Docker?
    - Hvor kan Docker bruges?
    - Hvordan kommer man igang?
- Hvordan bygger man applikationer og services med Docker
    - Docker-machine
    - Docker-compose (fig)
    - Docker-swarm     
    - privat Docker registry         
- Innovative anvendelser
    - docker-torrent
    - RunC
    - coreos
      - etcd
      - fleet
- Eksempler



# Indledning

   Anders Lehmann
   
   - Lektor EDE, Civilingeniør fra DTU
   - Ph.d. studerende
   - PyPy, B&O, DTU
   
   <img src="http://speed.pypy.org/static/images/logo.png" style="float:left"/>


   <img src="pypy_speed_2.6.png" width="50%" istyle="position:relaitve;top:-600px;left:50%"/>

# Hvad er Docker?

En ny virtualiserings teknologi

- Rapid deployment
- Kontrolleret udviklingsmiljø
- Skalerbarhed
- Lille overhead
- Kompromis mellem isolation og deling af resourcer

 Video
 
<iframe width="640" height="390" src="https://www.youtube.com/embed/ZzQfxoMFH0U" frameborder="0" allowfullscreen></iframe>



Demo
   IRSSI

# Docker historie

- pycon 2013
- Amazon, Google, Azure, Rackspace understøtter docker
- ?? Images tilgængelige på "docker hub"
- 1.0 udkom juni 2014
- 1.5 udkom i februar 2015

# Hvordan kan det lade sig gøre? 

- namespaces
- cgroups
- linuxcontainers (lxc)
- Aufs (Union filesystem)
- Docker daemon
- Docker networking

# AuFS

- Union File system
    - layers
    - tilføjelser, ændringer og sletninger
    - får udseende af et almindeligt
    
    
[eksempel]

Vis eksempel

/var/lib/aufs

# Docker Daemon

- Arbejds hesten
- eksekveres som root
- håndterer build, eksekvering, monitorering og stop af docker systemet
<img src="https://images.duckduckgo.com/iu/?u=https%3A%2F%2Fd262ilb51hltx0.cloudfront.net%2Fmax%2F800%2F1*9qFRI6SByU5fCrubBl7z_Q.jpeg&f=1" height="600px"/>

# Docker Networking

- Docker daemon bruger private network 172... til intern kommunikation
- De private network 10. og 192.168 kan bruges til host og applikations kommunikation

[Vis ifconfig i en container]

# Docker Networking

- Networking er uafhægig af Image
- Netværket konfigureres når containeren oprettes

# Hvordan kommer jeg igang?

Install docker

- Kitematic GUI (ikke linux)
- Docker toolbox (command line)
- apt-get/yum/rpm/whatever install docker/docker.io på linux
- https://www.docker.com/tryit/

Hent images fra Docker registry http://registry.hub.docker.com/


# Images/Containere

- Images er komplette virtualiserede applikationer
    * indeholder OS, biblioteker samt applikation
    * Kan indeholde data (antipattern)
- Containere er det (runtime)miljø, der oprettes for at køre et image
    * sørger for isolation 
    * kan dele OS og biblioteker
    * kan håndhæve resource begrænsninger

# Docker kommandoer

- Docker ps

    - Viser status for containere
    
    
- Docker run

    - Starter et image i en container
    
    
- Docker commit

    - Skaber et Image fra ændringer lavet i en container
    
    
- Docker build

    - Bygger et Image fra en Dockerfile
    
    
- Docker exec

    - Eksekverer en kommando i en container
    
    
- Docker logs

    - Viser stdout fra en container

# Dockerfile

- Opskrift for at bygge et Image
- Starter med et FROM 
- Opskriften udføres sekventielt
- Med RUN kommandoer kon operativ kommandoer udføres
- Med COPY kan kode kopieres ind fra host til Image
- VOLUME bruge til at knytte en host folder til Image


```
FROM debian:wheezy

# add our user and group first to make sure their IDs get assigned consistently, regardless of whatever dependencies get added
RUN groupadd -r redis && useradd -r -g redis redis

RUN apt-get update \
	&& apt-get install -y curl \
	&& rm -rf /var/lib/apt/lists/*

# grab gosu for easy step-down from root
RUN gpg --keyserver pool.sks-keyservers.net --recv-keys B42F6819007F00F88E364FD4036A9C25BF357DD4
RUN curl -o /usr/local/bin/gosu -SL "https://github.com/tianon/gosu/releases/download/1.2/gosu-$(dpkg --print-architecture)" \
	&& curl -o /usr/local/bin/gosu.asc -SL "https://github.com/tianon/gosu/releases/download/1.2/gosu-$(dpkg --print-architecture).asc" \
	&& gpg --verify /usr/local/bin/gosu.asc \
	&& rm /usr/local/bin/gosu.asc \
	&& chmod +x /usr/local/bin/gosu

ENV REDIS_VERSION 2.8.19
ENV REDIS_DOWNLOAD_URL http://download.redis.io/releases/redis-2.8.19.tar.gz
ENV REDIS_DOWNLOAD_SHA1 3e362f4770ac2fdbdce58a5aa951c1967e0facc8

# for redis-sentinel see: http://redis.io/topics/sentinel
RUN buildDeps='gcc libc6-dev make'; \
	set -x \
	&& apt-get update && apt-get install -y $buildDeps --no-install-recommends \
	&& rm -rf /var/lib/apt/lists/* \
	&& mkdir -p /usr/src/redis \
	&& curl -sSL "$REDIS_DOWNLOAD_URL" -o redis.tar.gz \
	&& echo "$REDIS_DOWNLOAD_SHA1 *redis.tar.gz" | sha1sum -c - \
	&& tar -xzf redis.tar.gz -C /usr/src/redis --strip-components=1 \
	&& rm redis.tar.gz \
	&& make -C /usr/src/redis \
	&& make -C /usr/src/redis install \
	&& rm -r /usr/src/redis \
	&& apt-get purge -y --auto-remove $buildDeps

RUN mkdir /data && chown redis:redis /data
VOLUME /data
WORKDIR /data

COPY docker-entrypoint.sh /entrypoint.sh
ENTRYPOINT ["/entrypoint.sh"]

EXPOSE 6379
CMD [ "redis-server" ]
```

# Docker run kommando

- Run kommandoen er meget kompleks
- Bruges til at fortælle Docker om det miljø Containeren skal køre i 

[vis docker help run]

- De vigtigste options er:
    - -d for daemonisering
    - -i for interaktiv
    - -t for terminal
    - --rm fjerner container når containeren slutter
    - -p map port fra host til container
    - --name giver containeren et specifikt navn
    - --link linker container med navngiven container
    - --cpu bestemmer andelen af cputid containeren må benytte
    - --memory til at begrænse hukommelses forbruget

# Tricks og værktøjer 

- Fig (docker-compose)
    [vis eksempel]
- docker-machine
- docker-swarm


- phusion/baseimage



# Seje applikationer 

- torrent-docker
    - https://github.com/mafintosh/torrent-docker
    
- builder
    - https://registry.hub.docker.com/u/tzolov/apache-spark-build-pipeline/
    - pypy
    
- tmpnb
    - https://github.com/jupyter/tmpnb


# Applikationer

https://www.youtube.com/watch?v=GsLZz8cZCzc

- IRSSI
- Chrome
- ...

# Docker-machine

- ensartet måde at håndtere Docker hosts
    - Localt
    - Virtualbox /VMWare Fusion
    - Digital ocean
    - AWS
    - ....
    

# Docker-compose

- Lav applikationer ved at sammen sætte microservices
- Scalér containere
    - web = 3 bliver til 3 containere med servicen "web"
- Specificer applikationen med en YAML fil



# Docker-compose

- Simpel deklarativ beskrivelse af services
- beskrivelse af interfaces
- Docker-compose beregner afhængigheder og bestemmer orden af opstart




# Docker-swarm

- Omdanner en gruppe af docker hosts til et cluster
- Kan benytte clusteret som en docker host



# Privat registry

- Findes som et Docker image !!!
- Tillader at dele images i en virksomhed eller lukket gruppe


# Demo

<img src="Slide2.png">

# Tak

Spørgsmål